In [22]:
#needed imports from Pandas, Numpy, Pickle, Matplot, Keras
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
from os import listdir
from keras.preprocessing import sequence
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import Adam
from keras.models import load_model
from keras.callbacks import ModelCheckpoint
import pprint
import os

os.environ['KMP_DUPLICATE_LIB_OK']='True'

if os.path.exists("best_model.pkl"): os.remove("best_model.pkl")

In [12]:
#loading raw data
path = 'MovementAAL/new_dataset/stream_0000'
sequences = list()

for i in range (1,9):
    file_path = path + str(i) + '.csv'
    df = pd.read_csv(file_path, header=0, usecols=["time", "delta0", "delta1", "delta2", "delta3"])
    values = df.values
    sequences.append(values)

path = 'MovementAAL/new_dataset/stream_000'

for i in range(10, 59):
    file_path = path + str(i) + '.csv'
    df = pd.read_csv(file_path, header=0, usecols=["time", "delta0", "delta1", "delta2", "delta3"])
    sequences.append(values)


In [6]:
#target values
targets = pd.read_csv('MovementAAL/new_dataset/MovementAAL_target.csv')
targets = targets.values[:,1]
print(targets)

[ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]


In [7]:
#Loading grouping
groups = pd.read_csv('MovementAAL/newgroups/MovementAAL_DatasetGroup.csv', header=0)
groups = groups.values[:,1]
print(groups)

[1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 1 1 1 1 1 1 1 1
 1 1 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3]


In [8]:
len_sequences = []
for one_seq in sequences:
    len_sequences.append(len(one_seq))
print(pd.Series(len_sequences).describe())

count    57.0
mean     30.0
std       0.0
min      30.0
25%      30.0
50%      30.0
75%      30.0
max      30.0
dtype: float64


In [13]:
#Padding the sequence with the values in last row to max length
to_pad = 30
new_seq = []
for one_seq in sequences:
    len_one_seq = len(one_seq)
    last_val = one_seq[-1]
    n = to_pad - len_one_seq
   
    to_concat = np.repeat(one_seq[-1], n).reshape(5, n).transpose()
    new_one_seq = np.concatenate([one_seq, to_concat])
    new_seq.append(new_one_seq)
final_seq = np.stack(new_seq)
print(final_seq)

[[[ 20  23  42  65  55]
  [ 32  29  44  66  62]
  [ 22  19  32  46  42]
  ...
  [ -3   1  -1  -1   2]
  [  4   3   1  -1   2]
  [  1   3   1   3   2]]

 [[ 18  17  33  48  44]
  [ 22  21  36  44  41]
  [ 18  19  26  35  40]
  ...
  [  1  -2   2   0   2]
  [ -1   1  -1  -1   1]
  [ -1   1  -1  -1   1]]

 [[ 29  28  54  73  66]
  [ 23  15  29  43  44]
  [ 17  14  24  33  37]
  ...
  [  1   6   2   1  -1]
  [  2  -2  -1   1   4]
  [  2  -2  -1   1   3]]

 ...

 [[ 25  25  51  79  73]
  [ 30  28  62 100  83]
  [ 71  72 105 158 139]
  ...
  [ -2   2   0   2   2]
  [  3   1   1   3   4]
  [  3   1   1   3   4]]

 [[ 25  25  51  79  73]
  [ 30  28  62 100  83]
  [ 71  72 105 158 139]
  ...
  [ -2   2   0   2   2]
  [  3   1   1   3   4]
  [  3   1   1   3   4]]

 [[ 25  25  51  79  73]
  [ 30  28  62 100  83]
  [ 71  72 105 158 139]
  ...
  [ -2   2   0   2   2]
  [  3   1   1   3   4]
  [  3   1   1   3   4]]]


In [15]:
#truncate the sequence to length 60
seq_len = 75
final_seq=sequence.pad_sequences(final_seq, maxlen=seq_len, padding='post', dtype='float', truncating='post')
print(final_seq)

[[[ 20.  23.  42.  65.  55.]
  [ 32.  29.  44.  66.  62.]
  [ 22.  19.  32.  46.  42.]
  ...
  [  0.   0.   0.   0.   0.]
  [  0.   0.   0.   0.   0.]
  [  0.   0.   0.   0.   0.]]

 [[ 18.  17.  33.  48.  44.]
  [ 22.  21.  36.  44.  41.]
  [ 18.  19.  26.  35.  40.]
  ...
  [  0.   0.   0.   0.   0.]
  [  0.   0.   0.   0.   0.]
  [  0.   0.   0.   0.   0.]]

 [[ 29.  28.  54.  73.  66.]
  [ 23.  15.  29.  43.  44.]
  [ 17.  14.  24.  33.  37.]
  ...
  [  0.   0.   0.   0.   0.]
  [  0.   0.   0.   0.   0.]
  [  0.   0.   0.   0.   0.]]

 ...

 [[ 25.  25.  51.  79.  73.]
  [ 30.  28.  62. 100.  83.]
  [ 71.  72. 105. 158. 139.]
  ...
  [  0.   0.   0.   0.   0.]
  [  0.   0.   0.   0.   0.]
  [  0.   0.   0.   0.   0.]]

 [[ 25.  25.  51.  79.  73.]
  [ 30.  28.  62. 100.  83.]
  [ 71.  72. 105. 158. 139.]
  ...
  [  0.   0.   0.   0.   0.]
  [  0.   0.   0.   0.   0.]
  [  0.   0.   0.   0.   0.]]

 [[ 25.  25.  51.  79.  73.]
  [ 30.  28.  62. 100.  83.]
  [ 71.  72. 105. 158. 139

In [17]:
#Training data based on group 2
train = [final_seq[i] for i in range(len(groups)) if (groups[i]==2)]
#validation data based on group 1
validation = [final_seq[i] for i in range(len(groups)) if groups[i]==1]
#test data based on group 3
test = [final_seq[i] for i in range(len(groups)-1) if groups[i]==3]

#train target based on group 2
train_target = [targets[i] for i in range(len(groups)) if (groups[i]==2)]
#validation target based on group 1
validation_target = [targets[i] for i in range(len(groups)) if groups[i]==1]
#test target based on group 3
test_target = [targets[i] for i in range(len(groups)) if groups[i]==3]

print(train)

[array([[ 25.,  25.,  51.,  79.,  73.],
       [ 30.,  28.,  62., 100.,  83.],
       [ 71.,  72., 105., 158., 139.],
       [ 71.,  72., 110., 158., 139.],
       [ 71.,  72., 110., 152., 143.],
       [ 71.,  72., 108., 150., 132.],
       [ 71.,  64.,  93., 136., 126.],
       [ 36.,  23.,  48.,  73.,  54.],
       [ 25.,  23.,  46.,  59.,  54.],
       [ 25.,  23.,  46.,  58.,  54.],
       [ 25.,  23.,  35.,  38.,  33.],
       [ 25.,  23.,  31.,  38.,  33.],
       [ 23.,  16.,  31.,  38.,  33.],
       [  7.,   7.,  19.,  14.,   9.],
       [  7.,   7.,  19.,  11.,   9.],
       [  7.,   7.,  19.,  11.,   9.],
       [  4.,   1.,  10.,   5.,   4.],
       [  4.,   1.,  10.,   5.,   4.],
       [  4.,   1.,  10.,   5.,   4.],
       [  6.,   2.,   3.,   5.,   6.],
       [  6.,   2.,   3.,   5.,   6.],
       [  6.,   2.,   3.,   5.,   6.],
       [  2.,   1.,   5.,   4.,   5.],
       [  2.,   1.,   5.,   4.,   5.],
       [  2.,   1.,   5.,   4.,   5.],
       [ -2.,   2.,   0.

In [18]:
#creating np.arrays for each dataset
train = np.array(train)
validation = np.array(validation)
test = np.array(test)

#training target data
train_target = np.array(train_target)
train_target = (train_target+1)/2

#validation target
validation_target = np.array(validation_target)
validation_target = (validation_target+1)/2

#test data and target test
test_target = np.array(test_target)
test_target = (test_target+1)/2

In [19]:
#adding the LSTM to the model and printing the summary
model = Sequential()
model.add(LSTM(120, input_shape=(seq_len, 5)))
model.add(Dense(1, activation='sigmoid'))
model.summary()
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 120)               60480     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 121       
Total params: 60,601
Trainable params: 60,601
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 120)               60480     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 121       
Total params: 60,601
Trainable params: 60,601
Non-trainable params: 0
_________________________________________________________________
None


In [20]:
adam = Adam(lr=0.001)
chk = ModelCheckpoint('best_model.pkl', monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
model.fit(train, train_target, epochs=200, batch_size=58, callbacks=[chk], validation_data=(validation,validation_target))


Train on 20 samples, validate on 20 samples
Epoch 1/200
20/20 [==============================] - 3s 151ms/step - loss: 0.6931 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.5000

Epoch 00001: val_accuracy improved from -inf to 0.50000, saving model to best_model.pkl
Epoch 2/200
20/20 [==============================] - 0s 13ms/step - loss: 0.6932 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.5000

Epoch 00002: val_accuracy did not improve from 0.50000
Epoch 3/200
20/20 [==============================] - 0s 13ms/step - loss: 0.6931 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.5000

Epoch 00003: val_accuracy did not improve from 0.50000
Epoch 4/200
20/20 [==============================] - 0s 13ms/step - loss: 0.6932 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.5000

Epoch 00004: val_accuracy did not improve from 0.50000
Epoch 5/200
20/20 [==============================] - 0s 13ms/step - loss: 0.6932 - accuracy: 0.5000 - val_loss: 0.6931 - val_accur

20/20 [==============================] - 0s 14ms/step - loss: 0.6931 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.5000

Epoch 00042: val_accuracy did not improve from 0.50000
Epoch 43/200
20/20 [==============================] - 0s 14ms/step - loss: 0.6931 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.5000

Epoch 00043: val_accuracy did not improve from 0.50000
Epoch 44/200
20/20 [==============================] - 0s 13ms/step - loss: 0.6931 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.5000

Epoch 00044: val_accuracy did not improve from 0.50000
Epoch 45/200
20/20 [==============================] - 0s 14ms/step - loss: 0.6931 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.5000

Epoch 00045: val_accuracy did not improve from 0.50000
Epoch 46/200
20/20 [==============================] - 0s 14ms/step - loss: 0.6931 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.5000

Epoch 00046: val_accuracy did not improve from 0.50000
Epoch 47/200
20/20

20/20 [==============================] - 0s 16ms/step - loss: 0.6931 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.8500

Epoch 00083: val_accuracy did not improve from 0.85000
Epoch 84/200
20/20 [==============================] - 0s 15ms/step - loss: 0.6931 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.5000

Epoch 00084: val_accuracy did not improve from 0.85000
Epoch 85/200
20/20 [==============================] - 0s 16ms/step - loss: 0.6931 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.5000

Epoch 00085: val_accuracy did not improve from 0.85000
Epoch 86/200
20/20 [==============================] - 0s 15ms/step - loss: 0.6931 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.5000

Epoch 00086: val_accuracy did not improve from 0.85000
Epoch 87/200
20/20 [==============================] - 0s 15ms/step - loss: 0.6931 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.5000

Epoch 00087: val_accuracy did not improve from 0.85000
Epoch 88/200
20/20


Epoch 00124: val_accuracy did not improve from 0.85000
Epoch 125/200
20/20 [==============================] - 0s 15ms/step - loss: 0.6931 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.5000

Epoch 00125: val_accuracy did not improve from 0.85000
Epoch 126/200
20/20 [==============================] - 0s 14ms/step - loss: 0.6931 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.8500

Epoch 00126: val_accuracy did not improve from 0.85000
Epoch 127/200
20/20 [==============================] - 0s 17ms/step - loss: 0.6931 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.8500

Epoch 00127: val_accuracy did not improve from 0.85000
Epoch 128/200
20/20 [==============================] - 0s 16ms/step - loss: 0.6931 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.8500

Epoch 00128: val_accuracy did not improve from 0.85000
Epoch 129/200
20/20 [==============================] - 0s 15ms/step - loss: 0.6931 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.8500



Epoch 00165: val_accuracy did not improve from 0.85000
Epoch 166/200
20/20 [==============================] - 0s 14ms/step - loss: 0.6931 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.5000

Epoch 00166: val_accuracy did not improve from 0.85000
Epoch 167/200
20/20 [==============================] - 0s 14ms/step - loss: 0.6931 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.5000

Epoch 00167: val_accuracy did not improve from 0.85000
Epoch 168/200
20/20 [==============================] - 0s 15ms/step - loss: 0.6931 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.5000

Epoch 00168: val_accuracy did not improve from 0.85000
Epoch 169/200
20/20 [==============================] - 0s 14ms/step - loss: 0.6931 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.8500

Epoch 00169: val_accuracy did not improve from 0.85000
Epoch 170/200
20/20 [==============================] - 0s 14ms/step - loss: 0.6931 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.8500


In [21]:
#loading the exported pkl model and testing the accuracy score
model = load_model('best_model.pkl')
from sklearn.metrics import accuracy_score
test_preds = model.predict_classes(test)
print(test)
print(test_preds)
print(test_target)
accuracy_score(test_target, test_preds)
print(accuracy_score(test_target, test_preds))

[[[ 25.  25.  51.  79.  73.]
  [ 30.  28.  62. 100.  83.]
  [ 71.  72. 105. 158. 139.]
  ...
  [  0.   0.   0.   0.   0.]
  [  0.   0.   0.   0.   0.]
  [  0.   0.   0.   0.   0.]]

 [[ 25.  25.  51.  79.  73.]
  [ 30.  28.  62. 100.  83.]
  [ 71.  72. 105. 158. 139.]
  ...
  [  0.   0.   0.   0.   0.]
  [  0.   0.   0.   0.   0.]
  [  0.   0.   0.   0.   0.]]

 [[ 25.  25.  51.  79.  73.]
  [ 30.  28.  62. 100.  83.]
  [ 71.  72. 105. 158. 139.]
  ...
  [  0.   0.   0.   0.   0.]
  [  0.   0.   0.   0.   0.]
  [  0.   0.   0.   0.   0.]]

 ...

 [[ 25.  25.  51.  79.  73.]
  [ 30.  28.  62. 100.  83.]
  [ 71.  72. 105. 158. 139.]
  ...
  [  0.   0.   0.   0.   0.]
  [  0.   0.   0.   0.   0.]
  [  0.   0.   0.   0.   0.]]

 [[ 25.  25.  51.  79.  73.]
  [ 30.  28.  62. 100.  83.]
  [ 71.  72. 105. 158. 139.]
  ...
  [  0.   0.   0.   0.   0.]
  [  0.   0.   0.   0.   0.]
  [  0.   0.   0.   0.   0.]]

 [[ 25.  25.  51.  79.  73.]
  [ 30.  28.  62. 100.  83.]
  [ 71.  72. 105. 158. 139

ValueError: Found input variables with inconsistent numbers of samples: [18, 17]